# Parameter - Audit

 [boxplot](http://www.math.wpi.edu/saspdf/stat/chap18.pdf)
 [git](http://wresch.github.io/2013/03/08/asinh-scales-in-ggplot2.html)
 [stack](https://stackoverflow.com/questions/37446064/i-need-ggplot-scale-x-log10-to-give-me-both-negative-and-positive-numbers-as-o)
 
[import](https://www.tutorialspoint.com/python/python_modules.htm)

[PATH](https://stackoverflow.com/questions/3701646/how-to-add-to-the-pythonpath-in-windows-so-it-finds-my-modules-packages)

[stats](https://ademos.people.uic.edu/Chapter13.html)

In [36]:
import numpy as np
from plotnine import *
import pandas as pd
from mizani.transforms import trans
from pathlib import Path

In [37]:
%matplotlib inline

In [38]:
def cleanIparm(dat_dir, pfile, dcol, df, st):
    df1 = pd.read_csv(dat_dir / pfile, header=0)
    ldcol = df1.loc[:, dcol]  # list with column dcol
    df.drop(ldcol, axis=1, inplace=True)  # delete params list in dataframes
    st.drop(ldcol, axis=0, inplace=True)
    return df, st

In [39]:
def cleanIparm2(df, st):
    import pandas as pd
    n = len(df.index)
    # ldcol = st.loc[]
    df.drop(st[st['StdDev'] == 0].index, axis=1, inplace=True)
    st.drop(st[st['StdDev'] == 0].index, inplace=True)
    df.drop(st[st['NoModeQty'] == 0].index, axis=1, inplace=True)
    st.drop(st[st['NoModeQty'] == 0].index, inplace=True)
    # parameter removal with high null percentage 
    df.drop(st[st['NaNQty'] > n * .2].index, axis=1, inplace=True)
    st.drop(st[st['NaNQty'] > n * .2].index, inplace=True)
    return df, st

In [40]:
def iqrcalc(q1, q3, n, std, mean):
    import numpy as np
    if .1 > mean > -.1:
        cv = 100 * std
    else:
        cv = 100 * std / abs(mean)
    return q3 + (1.58 * (q3 - q1) / np.sqrt(n)), q1 - (1.58 * (q3 - q1) / np.sqrt(n)), q3 - q1, cv

In [41]:
# case function for carrier selection. switcher is dictionary data type
def carrtext(carr):
    switcher = {
        0: 'Total',
        1: 'I:4387',
        2: 'II:9712',
        3: 'III:9685',
        4: 'IV:4364',
    }
    return switcher.get(carr, 'nothing') # 'nothing' if not found

In [42]:
# case function for carrier selection. switcher is dictionary data type
def carriers(carr):
    switcher = {
        0: 0,
        1: 4387,
        2: 9712,
        3: 9685,
        4: 4364,
    }
    return switcher.get(carr, 'nothing')

In [43]:
def par_audit(df):
    import functools
    n = len(df.index)  # row count
    # mode stored in columns
    modes = df.mode(dropna=False)
    # dummy rows delete
    modes = modes.dropna(subset=['Sector'])
    # dictionaries. data (count values diff from mode in modes) data1 (count of values = mode in modes)
    data = {col: (~df[col].isin(modes[col])).sum() for col in df.iloc[:, 0:].columns}
    data1 = {col: df[col].isin(modes[col]).sum() for col in df.iloc[:, 0:].columns}
    # st3 mode info
    st3 = pd.DataFrame.from_dict(data, orient='index', columns=['NoModeQty'])
    st3['ModeQty'] = pd.DataFrame.from_dict(data1, orient='index')
    st3['NoModePer'] = 100 * (st3['NoModeQty'] / (st3['ModeQty'] + st3['NoModeQty']))
    st3.index.name = 'parameter'
    st2 = modes.T
    st2.columns = ['Mode']
    st2.index.name = 'parameter'
    st1 = pd.DataFrame({'Vmin': df.min(), 'StdDev': df.std(), 'NaNQty': df.isnull().sum(axis=0), 'Mean': df.mean(),
                        'Q1': df.quantile(.25), 'Q3': df.quantile(.75), 'Median': df.quantile(.5)})
    st1[['Max', 'Min', 'IQR', 'CV']] = st1.apply(lambda row: iqrcalc(row['Q1'], row['Q3'], n, row['StdDev'],
                                                                     row['Mean']), axis=1, result_type='expand')
    st1.index.name = 'parameter'
    # df merge
    dfs = [st1, st2, st3]
    st4 = functools.reduce(lambda left, right: pd.merge(left, right, on='parameter'), dfs)
    st4.sort_values(by=['IQR', 'CV'], inplace=True, ascending=[False, False])
    return st4

In [44]:
class asinh_trans(trans):
    """
        asinh Transformation
        """

    @staticmethod
    def transform(y):
        y = np.asarray(y)
        return np.arcsinh(y)

    @staticmethod
    def inverse(y):
        y = np.asarray(y)
        return np.sinh(y)

In [45]:
def pntopd(file, figs, x, y, wi, he):
    from reportlab.pdfgen import canvas
    from reportlab.lib.pagesizes import A4, letter, landscape, portrait
    w, h = letter
    c = canvas.Canvas(str(file), pagesize=portrait(letter))
    for png in figs:
        c.drawImage(png, x, h - y, width=wi, height=he)
        c.showPage()
    c.save()

In [46]:
def par_aud(ruta, datb, tablas, tipo):
    import numpy as np
    from pyexcelerate import Workbook
    from pyexcelerate_to_excel import pyexcelerate_to_excel
    from datetime import date
    import sqlite3

    dat_dir = Path(ruta)
    db_path1 = dat_dir / datb
    conn = sqlite3.connect(db_path1)  # database connection
    c = conn.cursor()
    df1 = pd.read_csv(dat_dir / tablas)
    today = date.today()
    xls_file = tipo + today.strftime("%y%m%d") + ".xlsx"
    xls_path = dat_dir / xls_file  # xls file path-name
    wb = Workbook()  # pyexcelerate Workbook
    for index, row in df1.iterrows():  # table row iteration by audit2 column type
        line = row[tipo]
        if not pd.isna(row[tipo]):  # nan null values validation
            if line == 'WCEL':
                pnglist = []
                tit = today.strftime("%y%m%d") + '_ParameterAudit'
                xls_file = tit + ".xlsx"
                xls_path = dat_dir / xls_file
                pdf_file = tit + ".pdf"
                pdf_path = dat_dir / pdf_file
                n = 5  # carrier count - amount of graphs
                for i in range(0, n):  # loop for each carrier
                    carr = carriers(i)
                    cart = carrtext(i)
                    try:
                        if carr == 0:
                            df = pd.read_sql_query("select * from WCEL_FULL1;", conn, index_col=['WCELName', 'Prefijo'])
                        else:
                            df = pd.read_sql_query("select * from WCEL_FULL1 where (UARFCN = " + str(carr) + ");",
                                                   conn, index_col=['WCELName', 'Prefijo'])
                        st = par_audit(df)  # stats per parameter
                        df, st = cleanIparm(dat_dir, "ExParam.csv", "expar", df, st)  # info parameter removal
                        pyexcelerate_to_excel(wb, st, sheet_name='Carrier' + str(i), index=True)
                        df, st = cleanIparm2(df, st)  # standardized params and NaN>0.2*n removal
                        st.sort_values(by=['Median'], inplace=True, ascending=[False])  # for better visualization
                        st['counter'] = range(len(st))  # counter controls number of boxplots
                        st['counter'] = st['counter'].floordiv(10)  # split parameters in groups by 10
                        cols = ['StdDev', 'Mean', 'Median', 'Max', 'Min', 'CV']
                        st[cols] = st[cols].round(1)  # scales colums with 1 decimal digit
                        # concat info to put text in boxplots
                        st['concat'] = st['StdDev'].astype(str) + ', ' + st['NoModeQty'].astype(str)
                        ldcol = list(st.index)  # parameters to include in melt command
                        # Structuring df1 according to ‘tidy data‘ standard
                        df.reset_index(level=(0, 1), inplace=True)  # to use indexes in melt operation
                        df1 = df.melt(id_vars=['Prefijo'], value_vars=ldcol,  # WCELName is not used
                                      var_name='parameter', value_name='value')
                        st.reset_index(inplace=True)  # parameter from index to col
                        result = pd.merge(df1, st, on='parameter')  # merge by columns not by index
                        result['value_scale'] = np.arcsinh(result['value'])  # axis scaling
                        # graph code
                        custom_axis = theme(axis_text_x=element_text(color="grey", size=6, angle=90, hjust=.3),
                                            axis_text_y=element_text(color="grey", size=6),
                                            plot_title=element_text(size=25, face="bold"),
                                            axis_title=element_text(size=10),
                                            panel_spacing_x=1.6, panel_spacing_y=.45,
                                            # 2nd value number of rows and colunms
                                            figure_size=(5 * 4, 3.5 * 4)
                                            )
                        # ggplot code:value 'concat' is placed in coordinate (parameter, stddev)
                        my_plot = (ggplot(data=result, mapping=aes(x='parameter', y='value_scale')) + geom_boxplot() +
                                   geom_text(data=st, mapping=aes(x='parameter', y='StdDev', label='concat'),
                                             color='red', va='top', ha='left', size=7, nudge_x=.6, nudge_y=-1.5) +
                                   facet_wrap('counter', scales='free') + custom_axis +
                                   scale_y_continuous(trans=asinh_trans) + ylab("Values") + xlab("Parameters") +
                                   labs(title="Parameter Audit UMTS Carrier " + cart) + coord_flip())
                        pngname = "C" + str(carr) + ".png"
                        pngfile = dat_dir / pngname
                        my_plot.save(pngfile, width=20, height=10, dpi=300)
                        pnglist.append(pngfile)
                    except sqlite3.Error as error:  # sqlite error handling.
                        print('SQLite error: %s' % (' '.join(error.args)))
                        feedbk = tk.Label(top, text='SQLite error: %s' % (' '.join(error.args)))
                        feedbk.pack()
    wb.save(xls_path)
    pntopd(pdf_path, pnglist, 50, 550, 500, 500)
    c.close()
    conn.close()

In [47]:
par_aud("C:/SQLite", "20200522_sqlite.db", "tablasSQL.csv", "audit2")  # audit2 column from csv table file

C:\ProgramData\Anaconda3\lib\site-packages\plotnine\ggplot.py:727: PlotnineWarning: Saving 20 x 10 in image.
C:\ProgramData\Anaconda3\lib\site-packages\plotnine\ggplot.py:730: PlotnineWarning: Filename: C:\SQLite\C0.png
C:\ProgramData\Anaconda3\lib\site-packages\plotnine\utils.py:1246: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
C:\ProgramData\Anaconda3\lib\site-packages\plotnine\stats\stat_boxplot.py:86: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
C:\ProgramData\Anaconda3\lib\site-packages\plotnine\facets\facet.py:549: PlotnineWarning: If you need more space for the x-axis tick text use ... + theme(subplots_adjust={'wspace': 0.25}). Choose an appropriate value for 'wspace'.
C:\ProgramData\Anaconda3\lib\site-packages\plotnine\facets\facet.py:555: PlotnineWarning: If you need more space for the y-axis tick text use ... + theme(subplots_adj